In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib.pyplot as plt
# Model training

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV



In [4]:
from sklearn.model_selection import cross_val_score 
from sklearn.naive_bayes import GaussianNB 
from sklearn.linear_model import LogisticRegression 
from sklearn import tree 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

In [5]:
data_full = pd.read_csv('dataset_full.csv',na_values='?')
data_desease = pd.read_csv('dataset_desease.csv',na_values='?')


df_num_col = []
df_cat_col = []
for col in data_full.columns :
    if data_full[col].dtype == 'object':
        df_cat_col.append(col)
    else:
        df_num_col.append(col)
        
data_full_remplace = data_full.copy()

data_full_remplace = data_full_remplace.replace('yes',1)
data_full_remplace = data_full_remplace.replace('no',0)

data_full_remplace = data_full_remplace.replace('present',1)
data_full_remplace = data_full_remplace.replace('notpresent',0)

data_full_remplace = data_full_remplace.replace('normal',1)
data_full_remplace = data_full_remplace.replace('abnormal',0)

data_full_remplace = data_full_remplace.replace('good',1)
data_full_remplace = data_full_remplace.replace('poor',0)

data_full_remplace = data_full_remplace.replace('ckd',1)
data_full_remplace = data_full_remplace.replace('notckd',0)

for col in df_num_col :
    data_full_remplace[col]= data_full_remplace[col].fillna(data_full_remplace[col].mean())
    
# 1ere idée pour les models : drop la colonne Red_Blood_Cells et Pus_Cell qui contient 38% et 18% d'inconnues
drop_col = ['Red_Blood_Cells','Pus_Cell']
# les autres on remplace par la médiane
data_full_remplace_drop = data_full_remplace.copy()
data_full_remplace_drop=data_full_remplace_drop.drop(['Red_Blood_Cells','Pus_Cell'],axis=1)

for col in df_cat_col :
    if col != 'Red_Blood_Cells' and col != 'Pus_Cell' :
        data_full_remplace_drop[col]= data_full_remplace_drop[col].fillna(data_full_remplace_drop[col].median())



In [22]:
def preprocessing(dataset):
    df_num_col = []
    df_cat_col = []
    for col in dataset.columns :
        if dataset[col].dtype == 'object':
            df_cat_col.append(col)
        else:
            df_num_col.append(col)

    data_full_remplace = dataset.copy()

    data_full_remplace = data_full_remplace.replace('yes',1)
    data_full_remplace = data_full_remplace.replace('no',0)

    data_full_remplace = data_full_remplace.replace('present',1)
    data_full_remplace = data_full_remplace.replace('notpresent',0)

    data_full_remplace = data_full_remplace.replace('normal',1)
    data_full_remplace = data_full_remplace.replace('abnormal',0)

    data_full_remplace = data_full_remplace.replace('good',1)
    data_full_remplace = data_full_remplace.replace('poor',0)

    data_full_remplace = data_full_remplace.replace('ckd',1)
    data_full_remplace = data_full_remplace.replace('notckd',0)

    for col in df_num_col :
        data_full_remplace[col]= data_full_remplace[col].fillna(data_full_remplace[col].mean())

    # 1ere idée pour les models : drop la colonne Red_Blood_Cells et Pus_Cell qui contient 38% et 18% d'inconnues
    drop_col = ['Red_Blood_Cells','Pus_Cell']
    # les autres on remplace par la médiane
    data_full_remplace_drop = data_full_remplace.copy()
    data_full_remplace_drop=data_full_remplace_drop.drop(['Red_Blood_Cells','Pus_Cell'],axis=1)

    for col in df_cat_col :
        if col != 'Red_Blood_Cells' and col != 'Pus_Cell' :
            data_full_remplace_drop[col]= data_full_remplace_drop[col].fillna(data_full_remplace_drop[col].median())
    data_full_remplace_drop = data_full_remplace_drop.drop(['Class'],axis = 1)
    return(data_full_remplace_drop)
def return_pred(dataset):
    dataset.to_csv('out.csv',index=False)

In [6]:
X = data_full_remplace_drop.copy().drop(['Class'],axis=1)
y = data_full_remplace_drop['Class']
        
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X,y,random_state=0,train_size=0.8, test_size=0.2)     

In [7]:
lr = LogisticRegression(C= 78.47599703514607, max_iter= 2000, penalty='l2', solver= 'liblinear')
knn = KNeighborsClassifier(algorithm= 'auto', n_neighbors= 3, p= 1, weights= 'uniform')
svc = SVC(probability=True,C= 0.1, kernel= 'linear')
rf = RandomForestClassifier(bootstrap= False, max_depth= 10, min_samples_leaf= 2, min_samples_split= 2, n_estimators= 400)
xgb = XGBClassifier(colsample_bytree= 0.75, gamma= 0.5, learning_rate= 0.5, max_depth= None, min_child_weight= 0.01, n_estimators= 450, reg_alpha= 1, reg_lambda= 10, sampling_method= 'uniform', subsample= 0.65)


In [8]:
best_lr = lr.fit(train_X,train_y)
best_knn = knn.fit(train_X,train_y)
best_svc = svc.fit(train_X,train_y)
best_rf = rf.fit(train_X,train_y)
best_xgb = xgb.fit(train_X,train_y)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [9]:
voting = VotingClassifier(estimators=[('knn',best_knn),('rf',best_rf),('svc',best_svc),('lr',best_lr),('xgb',best_xgb)],voting='hard')
voting.fit(train_X,train_y)
preds = voting.predict(val_X)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [13]:
test_data = pd.read_csv('dataset_test.csv',na_values='?')
data_test = preprocessing(test_data)
preds = voting.predict(data_test)

In [17]:
test_data['Class']=preds

In [23]:
return_pred(test_data)